In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

from tqdm import tqdm

from catboost import CatBoostRegressor
import xgboost as xgb

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.model_selection import *
from sklearn.ensemble import StackingRegressor
from sklearn.impute import SimpleImputer

import optuna
from optuna.samplers import TPESampler

# Import Data

In [ ]:
df_train = pd.read_csv(r"../input/tabular-playground-series-sep-2021/train.csv", index_col=0)
df_test = pd.read_csv(r"../input/tabular-playground-series-sep-2021/test.csv", index_col=0)

In [ ]:
X = df_train.iloc[:, :-1]
y = df_train.iloc[:, -1]

X_test = df_test

DATA_FRAC = 0.001
df_train_frac = df_train.sample(frac=DATA_FRAC)
X_frac = df_train_frac.iloc[:, :-1]

In [ ]:
as_df = lambda x,df: pd.DataFrame(x, columns=df.columns, index=df.index)

def create_submission(model, X_test, filename):
    y_pred = model.predict_proba(X_test)
    #y_pred = np.round(y_pred)
    df_submission = pd.DataFrame(index = X_test.index )
    df_submission["claim"] = y_pred
    df_submission.to_csv(filename + ".csv")
    return df_submission

Some features might be categorical -> f29, f70, f75, ...

In [ ]:
categorical_features = ['f29', 'f65', 'f70', 'f74', 'f75']

for f in categorical_features:
    plt.figure()
    sns.histplot(data=df_train_frac, x=f)

In [ ]:
X["mean"] = X.mean(axis=1)
X_test["mean"] = X_test.mean(axis=1)

X["std"] = X.std(axis=1)
X_test["std"] = X_test.std(axis=1)

for i_cat, c in enumerate(categorical_features):
    X["cat_{}".format(i_cat)] = (X[c]>X[c].mean()).astype("int")
    X_test["cat_{}".format(i_cat)] = (X_test[c]>X[c].mean()).astype("int")

X_na["n_na"] = X.isna().sum(axis=1)
X_test_na["n_na"] = X_test.isna().sum(axis=1)

In [ ]:
xgb_param = {'max_depth': 5,
'n_estimators': 5000,
'eta': 0.008022742410676053,
'subsample': 0.7,
'colsample_bytree': 0.6000000000000001,
'colsample_bylevel': 0.6000000000000001,
'min_child_weight': 22.45859775663935,
'reg_lambda': 0.03496026308558376,
'reg_alpha': 4.793715821350734,
'gamma': 0.016465815248849734,
'use_label_encoder':False}

xgb_param['objective'] = 'binary:logistic'
xgb_param['tree_method']='gpu_hist'
xgb_param['predictor']='gpu_predictor'
xgb_param['n_jobs']=-1

print('Best parameters :')
display(xgb_param)
xgb_model = xgb.XGBClassifier(**xgb_param)

In [ ]:
xgb_model.fit(X, y, eval_metric='auc', eval_set=[(X,y)], verbose=500)
xgb_model.save_model("xgb_model.json")


create_submission(xgb_model, X_test, "xgb_submission")

In [ ]:
plt.bar(range(len(xgb_model.feature_importances_)), xgb_model.feature_importances_)
plt.show()

In [ ]:
X